# Run this code to deploy an agent

In [42]:
import vertexai
from virtual_nurse.agent import root_agent

PROJECT_ID = "virtual-nurse-450613"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://deploy-virtual-nurse"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

session = app.create_session(user_id="u_123")

In [43]:
session

Session(id='596e0704-c2e5-4bcd-b750-48d629e8af99', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1750445952.4070039)

In [44]:
app.list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='596e0704-c2e5-4bcd-b750-48d629e8af99', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1750445952.4070039)])

In [51]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="Scarlett, age 10, had a tonsillectomy 5 days ago and is currently visiting cousins out of town. The parent reports that the son's mouth smells really bad and he says it tastes gross. The patient does not report bleeding. The distance from the surgical hospital is 45 minutes and from the nearest hospital is 20 minutes. The patient last ate this afternoon and drank 1.5 cups of fluids in the past day. Lips are not chapped. The patient is not acting like themselves\u2014complaining a lot. Pain is not limiting intake. The patient is not experiencing nausea or vomiting and has no fevers. Pain is reported in the throat. There is no ear pain. The patient has a sore throat. Pain is managed with ibuprofen. Medications are not being alternated and they have not run out. Pain is not impacting intake. There is no bloody sputum. The throat has white patches only. The tongue/mouth has a yellowish coating. There is no breathing history and no cyanosis. The patient is annoyed and avoidant and not acting normal.",
):
    print(event)

{'content': {'parts': [{'text': 'Thank you for providing that information. Are they breathing okay?'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 12, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 12}], 'prompt_token_count': 2021, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 2021}], 'thoughts_token_count': 1466, 'total_token_count': 3499, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-88cee82d-f670-474d-a545-7ba5573eaaa4', 'author': 'triage_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'LsSs1CBW', 'timestamp': 1750449311.570063}


In [46]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-adk",
        "google-cloud-aiplatform[agent_engines]",
        "google-genai",
        "pydantic",
        "cloudpickle",  
    ],
    extra_packages=[
        "virtual_nurse",  
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.97.0', 'cloudpickle': '3.1.1'}
The final list of requirements: ['google-adk', 'google-cloud-aiplatform[agent_engines]', 'google-genai', 'pydantic', 'cloudpickle']
Using bucket deploy-virtual-nurse
Wrote to gs://deploy-virtual-nurse/agent_engine/agent_engine.pkl
Writing to gs://deploy-virtual-nurse/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://deploy-virtual-nurse/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/533873564462/locations/us-central1/reasoningEngines/8891627388522201088/operations/6072566264524439552
View progress and logs at https://console.cloud.google.com/logs/query?project=virtual-nurse-450613
AgentEngine created. Resource name: projects/533873564462/locations/us-central1/reasoningEngines/8891627388522201088
To use this AgentEngine in ano

In [47]:
remote_app.resource_name

'projects/533873564462/locations/us-central1/reasoningEngines/8891627388522201088'

In [50]:
remote_session = remote_app.create_session(user_id="u_123")
remote_session

FailedPrecondition: 400 Reasoning Engine Execution failed.
Please refer to our documentation (https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/troubleshooting/use) for checking logs and other troubleshooting tips.
Error Details: {"detail":"Agent Engine Error: An error occurred during invocation. Exception: Failed to create session.\nRequest Data: {'user_id': 'u_123'}"}

In [ ]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="what is Apple stock price?",
):
    print(event)